# Retrieval Optimizer Comparison

In the example, we are going to conduct pairwise tests of 3 embedding models and 5 retrieval methods.

## Embedding models

- sentence-transformers/all-MiniLM-L6-v2
- sentence-transformers/all-mpnet-base-v2
- openai/text-embedding-3-small

## Search methods

- pure BM25
- vector search
- hybrid (BM25 + vector)
- rerank (cross-encoder/ms-marco-MiniLM-L-6-v2)
- weighted_rrf

In [1]:
import sys
import os

def add_parent_path():
    # Get the current notebook directory
    current_dir = os.path.dirname(os.path.abspath(''))

    # Go up two directory levels (adjust the number as needed)
    parent_dir = os.path.abspath(os.path.join(current_dir, '../..'))

    # Add the parent directory to the Python path if it's not already there
    if parent_dir not in sys.path:
        sys.path.insert(0, parent_dir)
        print(f"Added {parent_dir} to Python path")


In [2]:
import os
from redis_retrieval_optimizer.grid_study import run_grid_study
from redis_retrieval_optimizer.corpus_processors import eval_beir
from dotenv import load_dotenv

# load environment variables containing necessary credentials
load_dotenv()

# until redis_retrieval_optimizer is released
add_parent_path()

redis_url = os.environ.get("REDIS_URL", "redis://localhost:6379/0")

metrics = run_grid_study(
    config_path="comparison_study_config.yaml",
    redis_url=redis_url,
    corpus_processor=eval_beir.process_corpus
)

/Users/robert.shelton/Library/Caches/pypoetry/virtualenvs/redis-retrieval-optimizer-OYoDaORV-py3.11/lib/python3.11/site-packages/beir/util.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


10:39:25 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Recreating: loading corpus from file
10:39:30 httpx INFO   HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
Running search method: bm25
Running search method: vector
Running search method: hybrid
Running search method: rerank
10:39:44 sentence_transformers.cross_encoder.CrossEncoder INFO   Use pytorch device: mps


Batches:   0%|          | 0/1 [00:00<?, ?it/s]0.00s - Debugger warning: It seems that frozen modules are being used, which may
0.00s - make the debugger miss breakpoints. Please pass -Xfrozen_modules=off
0.00s - to python to disable frozen modules.
0.00s - Note: Debugging will proceed. Set PYDEVD_DISABLE_FILE_VALIDATION=1 to disable this validation.
Batches: 100%|██████████| 1/1 [00:00<00:00,  7.47it/s]
ERROR:tornado.general:SEND Error: Host unreachable
Batches: 100%|██████████| 1/1 [00:00<00:00, 49.44it/s]


Running search method: weighted_rrf
Recreating index with new embedding model
10:41:58 redisvl.index.index INFO   Index already exists, overwriting.
If using multiple embedding models assuming there is a json version of corpus available.
Recreating: loading corpus from file
10:41:58 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: mps
10:41:58 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2


Batches: 100%|██████████| 1/1 [00:00<00:00,  7.77it/s]


...

10:42:56 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: mps
10:42:56 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


Batches: 100%|██████████| 1/1 [00:00<00:00, 27.47it/s]


Running search method: bm25
Running search method: vector
Running search method: hybrid
Running search method: rerank
10:43:04 sentence_transformers.cross_encoder.CrossEncoder INFO   Use pytorch device: mps


Batches: 100%|██████████| 1/1 [00:00<00:00, 29.93it/s]


Running search method: weighted_rrf


In [4]:
metrics.to_csv("comparison_study_results.csv", index=False)

In [8]:
metrics[["search_method", "model", "model_dim", 'total_indexing_time', "avg_query_time", "recall@k", "precision", "ndcg@k"]].sort_values(by="ndcg@k", ascending=False)

,search_method,model,model_dim,total_indexing_time,avg_query_time,recall@k,precision,ndcg@k
1,vector,text-embedding-3-small,1536,1.41785,0.009032,0.183617,0.286997,0.242556
4,weighted_rrf,text-embedding-3-small,1536,1.41785,0.013479,0.180653,0.272136,0.239084
2,hybrid,text-embedding-3-small,1536,1.41785,0.009755,0.183617,0.286997,0.238826
14,weighted_rrf,sentence-transformers/all-mpnet-base-v2,768,1.40007,0.012362,0.176480,0.261300,0.226050
3,rerank,text-embedding-3-small,1536,1.41785,0.400410,0.176467,0.266873,0.223536
9,weighted_rrf,sentence-transformers/all-MiniLM-L6-v2,384,1.11399,0.006921,0.167163,0.244272,0.214655
7,hybrid,sentence-transformers/all-MiniLM-L6-v2,384,1.11399,0.003396,0.154988,0.243344,0.203624
8,rerank,sentence-transformers/all-MiniLM-L6-v2,384,1.11399,0.104114,0.166115,0.254799,0.202444
6,vector,sentence-transformers/all-MiniLM-L6-v2,384,1.11399,0.003071,0.154988,0.243344,0.196586
13,rerank,sentence-transformers/all-mpnet-base-v2,768,1.40007,0.106512,0.162042,0.275527,0.196545


In [6]:
metrics.columns

Index(['search_method', 'total_indexing_time', 'avg_query_time', 'recall@k',
       'ndcg@k', 'f1@k', 'precision', 'ret_k', 'algorithm', 'ef_construction',
       'ef_runtime', 'm', 'distance_metric', 'vector_data_type', 'model',
       'model_dim'],
      dtype='object')